# Federated Linear Model with Scikit-learn

Before do the training, we need to setup NVFLARE

## Setup NVFLARE

Follow [Getting Started](https://nvflare.readthedocs.io/en/main/getting_started.html) to set up a virtual environment and install NVFLARE.

You can also follow this [notebook](https://github.com/NVIDIA/NVFlare/blob/main/examples/nvflare_setup.ipynb) to get set up.

> Make sure you have installed nvflare from **terminal** 




## Install requirements
assuming the current directory is 'higgs/stats'

In [ ]:
! pwd

In [ ]:
%pip install -r requirements.txt


## Prepare data
Please reference [prepare_higgs_data](../prepare_data.ipynb) notebooks. Pay attention to the current location. You need to switch "higgs" directory to run the data split.
    

Now we have our data prepared. we are ready to do the training

### Data Cleaning 

We noticed from time-to-time the Higgs dataset is making small changes which causing job to fail. so we need to do some clean up or skip certain rows. 
For example: certain floating number mistakenly add an alphabetical letter at some point of time. This may have already fixed by UCI. 

As of today: 2023-11-21, the dataset we downloaded seems to have some value that can't convert to floating point. value such as "0.000000000000000000e+00.1"

Here we notice this seems to be at 1st row of the files. so we are going to use skip_rows during data loading to avoid this. 


## Scikit-learn
This example shows how to use [NVIDIA FLARE](https://nvflare.readthedocs.io/en/main/index.html) on tabular data.
It uses [Scikit-learn](https://scikit-learn.org/), a widely used 
open-source machine learning library that supports supervised and unsupervised learning.


### Federated Linear Model
Here we use of [linear classifiers with SGD training](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) in a federated scenario.
Under this setting, federated learning can be formulated as a [FedAvg](https://arxiv.org/abs/1602.05629) process with local training that each client optimizes the local model starting from global parameters with SGD. 

This can be achieved by setting the `warm_start` flag of SGDClassifier to `True` in order to allow repeated fitting of the classifiers to the local data. 

Let's look at the code see how we convert the traditional Linear model to the federated model


In [ ]:
!pwd


In [ ]:
!cat code/sgd_fl.py

The code is pretty much like standard scikit learn traing program

#### load data

We first load the features from the header file: 
    
```
    site_name = flare.get_site_name()
    feature_data_path = f"{data_root_dir}/{site_name}_header.csv"
    features = load_features(feature_data_path)
    n_features = len(features) -1

    data_path = f"{data_root_dir}/{site_name}.csv"
    data = load_data(data_path=data_path, data_features=features, test_size=test_size, skip_rows=skip_rows)

```

then load the data from the main csv file, then transform the data and split the training and test data based on the test_size provided.  

```
    data = to_dataset_tuple(data)
    dataset = transform_data(data)
    x_train, y_train, train_size = dataset["train"]
    x_test, y_test, test_size = dataset["test"]

```

The part that's specific to Federated Learning is in the following codes

```
# (1) import nvflare client API
from nvflare import client as flare

```
```
# (2) initializes NVFlare client API
    flare.init()

    site_name = flare.get_site_name()
    
```
    
These few lines, import NVFLARE Client API and initialize it, then use the API to find the site_name (such as site-1, site-2 etc.). With the site-name, we can construct the site-specific 
data path such as

```
    feature_data_path = f"{data_root_dir}/{site_name}_header.csv"

    data_path = f"{data_root_dir}/{site_name}.csv"
```

#### Training 

In the standard traditional scikit learn, we would construct the model such as
```
  model = SGDClassifier(...) 
```
then call model.fit(...)
```
  model.fit(x_train, y_train)

  accuracy, report = evaluate_model(x_test, model, y_test)

```

with federated learning, using FLARE Client API, we have to make a few changes
* 1) we are not only training in local iterations, but also global rounds, we need to keep the program running until we reached to the totoal number of rounds 
  
  ```
      while flare.is_running():
          ... rest of code
  
  ```
  
* 2) Unlike traditional machine learning, we have now have more than one clients/sites participating the training. To ensure every site starts with the same model parameters, we use server to broadcase the initial model parameters to every sites at the first round ( current_round = 0). 

* 3) We will need to use FLARE client API to receive global model and find out the global parameters

```
     # (3) receives FLModel from NVFlare
        input_model = flare.receive()
        global_params = input_model.params
        curr_round = input_model.current_round
```

```
      if curr_round == 0:
            # (4) initialize model with global_param
            # and set to all zero
            fit_intercept = bool(global_params["fit_intercept"])
            model = SGDClassifier(
                loss=global_params["loss"],
                penalty=global_params["penalty"],
                fit_intercept=fit_intercept,
                learning_rate=global_params["learning_rate"],
                eta0=global_params["eta0"],
                max_iter=1,
                warm_start=True,
                random_state=random_state,
            )
        ....
```
* 4) if it is not the first round, we need to use the global model to update the local model before training the next round. For Scikit-learn leaner SDG model, we simply update coeffient and intercept. 

```
         # (5) update model based on global parameters
            # the model has warm_start, so these parameters will be used in initialize the training
            if "coef" in global_params:
                model.coef_ = global_params["coef"]
            if model.fit_intercept and "intercept" in global_params:
                model.intercept_ = global_params["intercept"]
```

* 5) to make sure we have the best global model, we need to evaluate the global model using the local data

```
   # (6) evaluate global model first.
        global_accuracy, global_report = evaluate_model(x_test, model, y_test)
```
* 6) finally we do the training as before.

```
        # Train the model on the training set
        model.fit(x_train, y_train)
        
        accuracy, report = evaluate_model(x_test, model, y_test)

        # Print the results
        print(f"local model Accuracy: {accuracy:.2f}")
        print("local model Classification Report:\n", report)
        
```

* 7) we need the new training result (coeffient and intercept) back to the global model for aggregation, to do that, we have the following code

```
        # (7) construct trained FL model
        params = {"coef": model.coef_, "intercept": model.intercept_}
        metrics = {"accuracy": global_accuracy, "report": global_report}
        output_model = flare.FLModel(params=params, metrics=metrics)

        # (8) send model back to NVFlare
        flare.send(output_model)
```

## Prepare Job  

Now, we have the code, we need to prepare job folder with configurations to run in NVFLARE. To do this, we can leveage the job template for scikit learn. First look at the the available job templates

In [ ]:
! nvflare job list_templates

the sklarn_linear is the one we need. 

In [2]:
! nvflare job create -j /tmp/nvflare/jobs/sklearn_sgd -force -w sklearn_linear \
-sd code \
-f config_fed_client.conf  app_script="sgd_fl.py" app_config="--data_root_dir /tmp/nvflare/dataset/output --test_size 0.2" \
-f config_fed_server.conf  max_iter=1000 num_rounds=2


The following are the variables you can change in the template

---------------------------------------------------------------------------------------------------------------------------------------
                                                                                                                                       
  job folder: /tmp/nvflare/jobs/sklearn_sgd                                                                                              
                                                                                                                                       
---------------------------------------------------------------------------------------------------------------------------------------
  file_name                      var_name                       value                               component                          
---------------------------------------------------------------------------------------------------------------------

In [ ]:
! cat /tmp/nvflare/jobs/sklearn_sgd/app/config/config_fed_client.conf

In [ ]:
! tree  /tmp/nvflare/jobs/sklearn_sgd

>Note 
 we use skip_rows = 0 to skip 1st row. We could skip_rows = [0, 3] to skip first and 4th rows.



## Run job in simulator

We use the simulator to run this job

In [3]:
! nvflare simulator  /tmp/nvflare/jobs/sklearn_sgd   -w /tmp/nvflare/sklearn_sgd  -n 3 -t 3

2023-11-22 11:05:51,043 - SimulatorRunner - INFO - Create the Simulator Server.
2023-11-22 11:05:51,044 - CoreCell - INFO - server: creating listener on tcp://0:35939
2023-11-22 11:05:51,057 - CoreCell - INFO - server: created backbone external listener for tcp://0:35939
2023-11-22 11:05:51,057 - ConnectorManager - INFO - 33909: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-11-22 11:05:51,058 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:33285] is starting
2023-11-22 11:05:51,558 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:33285
2023-11-22 11:05:51,559 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:35939] is starting
2023-11-22 11:05:51,608 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 37499
2023-11-22 11:05:51,608 - SimulatorRunner - INFO - Deploy the Apps.
2023-11-22 11:05:51,612 - SimulatorRunner - INFO - Create

In [ ]:
!tree  /tmp/nvflare/sklearn_sgd 

In [ ]:
!cat /tmp/nvflare/sklearn_sgd/simulate_job/log.txt